# Support Vector Machine Model

In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%matplotlib inline

In [5]:
data = np.load("./data/data_pca_50_y_mean.pickle.npz")
data.files

['arr_0', 'arr_1', 'arr_2']

In [6]:
X = data["arr_0"]
y = data["arr_1"]
mean = data["arr_2"]

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((4370, 50), (1093, 50), (4370,), (1093,))

Trains machine learning model.

In [9]:
from sklearn.svm import SVC

model = SVC(C=1.0, kernel="rbf", gamma= 0.01)
model.fit(x_train, y_train)
print("Model trained successfully!")

Model trained successfully!


Scores.

In [11]:
model.score(x_train, y_train), model.score(x_test, y_test)

(0.8606407322654462, 0.8005489478499542)